In [46]:
import random
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
from scipy.stats import linregress

In [2]:
df = pd.read_csv('ReviewsEN.csv')

In [3]:
print("-1 :", df['sentiment'].value_counts()[-1])
print("0 :", df['sentiment'].value_counts()[0])
print("1 :", df['sentiment'].value_counts()[1])

-1 : 870
0 : 639
1 : 1518


In [4]:
# Replace values in pandas DataFrame.
df['sentiment'] = df['sentiment'].replace([1], 2)
df['sentiment'] = df['sentiment'].replace([0], 1)
df['sentiment'] = df['sentiment'].replace([-1], 0)

In [5]:
# Apply lower function
df['reviews'] = df['reviews'].apply(str.lower)
print(df)

      sentiment                                            reviews
0             1  teacher are punctual but they should also give...
1             2                                               good
2             2  excellent lectures are delivered by teachers a...
3             2  teachers give us all the information required ...
4             2                                                yes
...         ...                                                ...
3022          2        lecturers provide clear enough explanations
3023          2            lecturer's assessment is very objective
3024          0              lecturers give very good explanations
3025          0        lecturers often provoke discussion in class
3026          2         lecturers provide material in a boring way

[3027 rows x 2 columns]


In [28]:
# Global Variables

EMBEDDING_DIM = 100
MAXLEN = 16
TRUNCATING = 'post'
PADDING = 'post'
OOV_TOKEN = "<OOV>"
MAX_EXAMPLES = 3000
TRAINING_SPLIT = 0.9

In [30]:
import random

random.seed(42)

# Get the indices of the DataFrame
indices = df.index.tolist()

# Perform random sampling on the indices
selected_indices = random.sample(indices, MAX_EXAMPLES)

# Select the corresponding sentences and labels based on the sampled indices
sentences = df.loc[selected_indices, 'reviews']
labels = df.loc[selected_indices, 'sentiment']

print(f"There are {len(sentences)} sentences and {len(labels)} labels after random sampling\n")


There are 3000 sentences and 3000 labels after random sampling



# Training - Validation Split

In [31]:
def train_val_split(sentences, labels, training_split):
    ### START CODE HERE
    
    # Compute the number of sentences that will be used for training (should be an integer)
    train_size = int(len(sentences)*training_split)

    # Split the sentences and labels into train/validation splits
    train_sentences = sentences[:train_size]
    train_labels = labels[:train_size]

    validation_sentences = sentences[train_size:]
    validation_labels = labels[train_size:]
    
    ### END CODE HERE
    
    return train_sentences, validation_sentences, train_labels, validation_labels

In [32]:
train_sentences, val_sentences, train_labels, val_labels = train_val_split(sentences, labels, TRAINING_SPLIT)

print(f"There are {len(train_sentences)} sentences for training.\n")
print(f"There are {len(train_labels)} labels for training.\n")
print(f"There are {len(val_sentences)} sentences for validation.\n")
print(f"There are {len(val_labels)} labels for validation.")

There are 2700 sentences for training.

There are 2700 labels for training.

There are 300 sentences for validation.

There are 300 labels for validation.


# Tokenization - Sequences, Truncating, and Padding

In [33]:
# GRADED FUNCTION: fit_tokenizer
def fit_tokenizer(train_sentences, oov_token):
    ### START CODE HERE
    
    # Instantiate the Tokenizer class, passing in the correct values for oov_token
    tokenizer = Tokenizer(oov_token = OOV_TOKEN)
    
    # Fit the tokenizer to the training sentences
    tokenizer.fit_on_texts(train_sentences)
    
    ### END CODE HERE
    
    return tokenizer

In [56]:
# Test your function
tokenizer = fit_tokenizer(train_sentences, OOV_TOKEN)

word_index = tokenizer.word_index
VOCAB_SIZE = len(word_index)

print(f"Vocabulary contains {VOCAB_SIZE} words\n")
print("<OOV> token included in vocabulary" if "<OOV>" in word_index else "<OOV> token NOT included in vocabulary")
print(f"\nindex of word 'i' should be {word_index['i']}")
word_index

Vocabulary contains 3691 words

<OOV> token included in vocabulary

index of word 'i' should be 1139


{'<OOV>': 1,
 'class': 2,
 'professor': 3,
 'students': 4,
 'course': 5,
 'material': 6,
 'easy': 7,
 'teacher': 8,
 'understand': 9,
 'learning': 10,
 'lectures': 11,
 'help': 12,
 'teaching': 13,
 'hard': 14,
 'helpful': 15,
 'lecturers': 16,
 'questions': 17,
 'lot': 18,
 'assignments': 19,
 'difficult': 20,
 'professors': 21,
 'subject': 22,
 'time': 23,
 'exams': 24,
 'makes': 25,
 'exam': 26,
 'tests': 27,
 'learn': 28,
 'lecture': 29,
 'nice': 30,
 'provide': 31,
 'study': 32,
 'knowledge': 33,
 'recommend': 34,
 'math': 35,
 'classes': 36,
 'sometimes': 37,
 'understanding': 38,
 'homework': 39,
 'materials': 40,
 'final': 41,
 'test': 42,
 'knowledgeable': 43,
 'grading': 44,
 'extra': 45,
 'experience': 46,
 'boring': 47,
 'bad': 48,
 'engaging': 49,
 'jokes': 50,
 'credit': 51,
 'content': 52,
 'university': 53,
 'concepts': 54,
 'read': 55,
 'teachers': 56,
 'pretty': 57,
 'guy': 58,
 'fun': 59,
 'grade': 60,
 'fair': 61,
 'prof': 62,
 'funny': 63,
 'lab': 64,
 'practical':

In [42]:
def seq_pad_and_trunc(sentences, tokenizer, padding, truncating, maxlen):
    ### START CODE HERE
       
    # Convert sentences to sequences
    sequences = tokenizer.texts_to_sequences(sentences)
    
    # Pad the sequences using the correct padding, truncating and maxlen
    pad_trunc_sequences = pad_sequences(sequences, maxlen= MAXLEN, padding = PADDING, truncating = TRUNCATING)
    
    ### END CODE HERE
    
    return pad_trunc_sequences

In [43]:
train_pad_trunc_seq = seq_pad_and_trunc(train_sentences, tokenizer, PADDING, TRUNCATING, MAXLEN)
val_pad_trunc_seq = seq_pad_and_trunc(val_sentences, tokenizer, PADDING, TRUNCATING, MAXLEN)

print(f"Padded and truncated training sequences have shape: {train_pad_trunc_seq.shape}\n")
print(f"Padded and truncated validation sequences have shape: {val_pad_trunc_seq.shape}")

Padded and truncated training sequences have shape: (2700, 16)

Padded and truncated validation sequences have shape: (300, 16)


In [44]:
train_labels = np.array(train_labels)
val_labels = np.array(val_labels)

# Using pre-defined Embeddings

In [47]:
# Define path to file containing the embeddings
GLOVE_FILE = 'glove.6B.100d.txt'

# Initialize an empty embeddings index dictionary
GLOVE_EMBEDDINGS = {}

# Read file and fill GLOVE_EMBEDDINGS with its contents
with open(GLOVE_FILE) as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        GLOVE_EMBEDDINGS[word] = coefs

# Represent the words in your vocabulary using the embeddings

In [48]:
# Initialize an empty numpy array with the appropriate size
EMBEDDINGS_MATRIX = np.zeros((VOCAB_SIZE+1, EMBEDDING_DIM))

# Iterate all of the words in the vocabulary and if the vector representation for 
# each word exists within GloVe's representations, save it in the EMBEDDINGS_MATRIX array
for word, i in word_index.items():
    embedding_vector = GLOVE_EMBEDDINGS.get(word)
    if embedding_vector is not None:
        EMBEDDINGS_MATRIX[i] = embedding_vector

# Define a model that does not overfit

In [54]:
def create_model(vocab_size, embedding_dim, maxlen, embeddings_matrix):
    model = tf.keras.Sequential([ 
        # This is how you need to set the Embedding layer when using pre-trained embeddings
        tf.keras.layers.Embedding(vocab_size+1, embedding_dim, input_length=maxlen, weights=[embeddings_matrix], trainable=False),
        tf.keras.layers.Conv1D(32, 5, activation='relu'),
        tf.keras.layers.GlobalMaxPooling1D(),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(3, activation='softmax'),
    ])
    
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy']) 
    return model

In [55]:
model = create_model(VOCAB_SIZE, EMBEDDING_DIM, MAXLEN, EMBEDDINGS_MATRIX)

# Train the model and save the training history
history = model.fit(train_pad_trunc_seq, train_labels, epochs=200, validation_data=(val_pad_trunc_seq, val_labels))

Epoch 1/200
85/85 [==============================] - 2s 4ms/step - loss: 1.0282 - accuracy: 0.5096 - val_loss: 0.9825 - val_accuracy: 0.6033
Epoch 2/200
85/85 [==============================] - 0s 3ms/step - loss: 0.9146 - accuracy: 0.5863 - val_loss: 0.8944 - val_accuracy: 0.6333
Epoch 3/200
85/85 [==============================] - 0s 3ms/step - loss: 0.8277 - accuracy: 0.6370 - val_loss: 0.8445 - val_accuracy: 0.6400
Epoch 4/200
85/85 [==============================] - 0s 3ms/step - loss: 0.7538 - accuracy: 0.6793 - val_loss: 0.8082 - val_accuracy: 0.6433
Epoch 5/200
85/85 [==============================] - 0s 3ms/step - loss: 0.6838 - accuracy: 0.7163 - val_loss: 0.7990 - val_accuracy: 0.6667
Epoch 6/200
85/85 [==============================] - 0s 3ms/step - loss: 0.6372 - accuracy: 0.7315 - val_loss: 0.7995 - val_accuracy: 0.6567
Epoch 7/200
85/85 [==============================] - 0s 3ms/step - loss: 0.5712 - accuracy: 0.7678 - val_loss: 0.7903 - val_accuracy: 0.6767
Epoch 8/200
8

In [62]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score

# Make predictions on the validation set
val_predictions = model.predict(val_pad_trunc_seq)
val_predicted_labels = np.argmax(val_predictions, axis=1)

# Calculate precision, recall, and F1-score
precision = precision_score(val_labels, val_predicted_labels, average='weighted')
recall = recall_score(val_labels, val_predicted_labels, average='weighted')
f1 = f1_score(val_labels, val_predicted_labels, average='weighted')

print("Precision: ", precision)
print("Recall: ", recall)
print("F1-score: ", f1)


10/10 [==============================] - 0s 2ms/step
Precision:  0.6397891558654842
Recall:  0.65
F1-score:  0.6428606440581361


Based on the evaluation metrics, our model performs relatively well with relatively high values of precision, recall, and F1-score

# Stopwords

In [6]:
# opening the file in read mode
my_file = open("stopwords.txt", "r")
  
# reading the file
data = my_file.read()
  
# replacing end splitting the text 
# when newline ('\n') is seen.
stopwords_data = data.split("\n")
print(stopwords_data)
my_file.close()

['a', 'about', 'above', 'across', 'after', 'again', 'against', 'all', 'almost', 'alone', 'along', 'already', 'also', 'although', 'always', 'among', 'an', 'and', 'another', 'any', 'anybody', 'anyone', 'anything', 'anywhere', 'are', 'area', 'areas', 'around', 'as', 'ask', 'asked', 'asking', 'asks', 'at', 'away', 'b', 'back', 'backed', 'backing', 'backs', 'be', 'became', 'because', 'become', 'becomes', 'been', 'before', 'began', 'behind', 'being', 'beings', 'best', 'better', 'between', 'big', 'both', 'but', 'by', 'c', 'came', 'can', 'cannot', 'case', 'cases', 'certain', 'certainly', 'clear', 'clearly', 'come', 'could', 'd', 'did', 'differ', 'different', 'differently', 'do', 'does', 'done', 'down', 'down', 'downed', 'downing', 'downs', 'during', 'e', 'each', 'early', 'either', 'end', 'ended', 'ending', 'ends', 'enough', 'even', 'evenly', 'ever', 'every', 'everybody', 'everyone', 'everything', 'everywhere', 'f', 'face', 'faces', 'fact', 'facts', 'far', 'felt', 'few', 'find', 'finds', 'first

In [7]:
import string

def remove_punctuation(text):
    # Remove punctuation marks
    translator = str.maketrans('', '', string.punctuation)
    text_without_punctuation = text.translate(translator)
    return text_without_punctuation

In [8]:
cleaned_text = []
for i in range(len(df)):
  cleaned_text.append(remove_punctuation(df['reviews'][i]))

df['reviews'] = cleaned_text
df['reviews'].values

array(['teacher are punctual but they should also give us the some practical knowledge other than theortical',
       'good',
       'excellent lectures are delivered by teachers and all teachers are very punctual',
       ..., 'lecturers give very good explanations',
       'lecturers often provoke discussion in class',
       'lecturers provide material in a boring way'], dtype=object)

In [9]:
def remove_stopwords(sentence, data):
    # List of stopwords

    stopwords = data + ["a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]
    numbers_stopwords = ["1", "2", "3", "4","5","6","7","8","9","10",
                         "one", "two","three","four","five","098"]
    more_words = ["didn't", "don't", "dont", "didnt", "it", "doesnt", "doesn't", "hw","won't","lpu","weren't","mr","mcq","shes",
                  "shes","india","in","hes","shes","me", "dr", "nlandu", "ko","it","1st", "omr", "ha", "upto","ca", "soo", "cd", "ive","po","cse", "chem", "un","of",
                  "mte", "omr","mte's","ca's","ete's","jnv","ip","sir","its","wks","prob","python","java","lattc","ol","ived","elsewhere", "mother","wouldnt","car",
                  "si", "sat","we","home","hot","god","ice","money's","money","even","about","thats", "wks", "thurs", "months", "sir", "go", "jnv", "ip", "today", "today's", "linux", "github",
                  "lt", "ums", "superb", "at", "cgpa","ques", "brain's", "mcqs", "ve", "say", "pc", "viva", "after", "before", "draw", "asst", "only", "rich", "never", "went", "pcs", "gk", "one's",
                  "co", "duty", "gona", "attendnce","same", "that's", "hahahah", "ad's", "university's", "relly", "build", "cricket", "said", "hall", "profs", "guy's", "can", "along", "archieve", "bag",
                  "part", "master", "push", "or", "add", "were", "virginia","human", "bless", "clean", "count", "onlineopen", "ounce", "brushing", "zero", "mail", "fys", "lowell", "stets", "untill", "until",
                  "prep", "appears", "giulia", "yuk", "memo", "ton", "110q", "unit", "80","re","by","order","fob", "sit", "from","art", "org", "4d", "3d", "cinema", "iii", "cal", "both", "sundays", "todays", "ad",
                  "yoursel","yourself", "kiss", "it'll", "obayani's", "anal", "pgs", "csci", "hw", "more", "able", "lecturer", "lecturer's", "student", "stundet's", "it", "want", "you","he's", "she's"]
    more =  [
    'a', 'about', 'above', 'after', 'again', 'against', "ain't", 'all', 'am', 'an', 'and', 'any', 'are', 'aren\'t', 'as', 
    'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'can\'t', 'cannot', 
    'could', 'couldn\'t', 'did', 'didn\'t', 'do', 'does', 'doesn\'t', 'doing', 'don\'t', 'down', 'during', 'each', 'few', 
    'for', 'from', 'further', 'had', 'hadn\'t', 'has', 'hasn\'t', 'have', 'haven\'t', 'having', 'he', 'he\'d', 'he\'ll', 
    'he\'s', 'her', 'here', 'here\'s', 'hers', 'herself', 'him', 'himself', 'his', 'how', 'how\'s', 'i', 'i\'d', 'i\'ll', 
    'i\'m', 'i\'ve', 'if', 'in', 'into', 'is', 'isn\'t', 'it', 'it\'s', 'its', 'itself', 'let\'s', 'me', 'more', 'most', 
    'mustn\'t', 'my', 'myself', 'no', 'nor', 'not', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'ought', 'our', 
    'ours', 'ourselves', 'out', 'over', 'own', 'same', 'shan\'t', 'she', 'she\'d', 'she\'ll', 'she\'s', 'should', 
    'shouldn\'t', 'so', 'some', 'such', 'than', 'that', 'that\'s', 'the', 'their', 'theirs', 'them', 'themselves', 
    'then', 'there', 'there\'s', 'these', 'they', 'they\'d', 'they\'ll', 'they\'re', 'they\'ve', 'this', 'those', 
    'through', 'to', 'too', 'under', 'until', 'up', 'very', 'was', 'wasn\'t', 'we', 'we\'d', 'we\'ll', 'we\'re', 
    'we\'ve', 'were', 'weren\'t', 'what', 'what\'s', 'when', 'when\'s', 'where', 'where\'s', 'which', 'while', 'who', 
    'who\'s', 'whom', 'why', 'why\'s', 'with', 'won\'t', 'would', 'wouldn\'t', 'you', 'you\'d', 'you\'ll', 'you\'re', 
    'you\'ve', 'your', 'yours', 'yourself', 'yourselves']

    final_stopwords = stopwords + numbers_stopwords + more_words + more
    # Sentence converted to lowercase-only
    sentence = sentence.lower()
    
    ### START CODE HERE
    words = sentence.split()
    tempWords = []
    for i in words:
        if i not in final_stopwords:
            tempWords.append(i)
            sentence = ' '.join(tempWords)
    
    ### END CODE HERE
    return sentence

In [10]:
len(df)
df['reviews'].values

array(['teacher are punctual but they should also give us the some practical knowledge other than theortical',
       'good',
       'excellent lectures are delivered by teachers and all teachers are very punctual',
       ..., 'lecturers give very good explanations',
       'lecturers often provoke discussion in class',
       'lecturers provide material in a boring way'], dtype=object)

In [11]:
cKomen = []
for i in range(len(df)):
  cKomen.append(remove_stopwords(df['reviews'][i], stopwords_data))

df['reviews'] = cKomen

In [15]:
# HyperParameter
vocab_size = 1000
embedding_dim = 32
max_length = 64
trunc_type = 'post'
oov_tok = '<OOV>'
padding_type = 'post'

In [17]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)
testing_sentences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sentences, maxlen=max_length)